In [17]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Activation,Convolution2D,Dense,Dropout,MaxPooling2D,Input,Flatten
from keras.utils import np_utils
from keras.models import Sequential,Input
import pandas as pd
import operator

In [4]:
df = pd.read_csv('./train.csv')
test_file = pd.read_csv('./test.csv')

In [5]:
df = df.values
test_file = test_file.values

In [6]:
split = int(0.8*df.shape[0])
train_x = df[:split,1:]
train_y = df[:split,0]
test_x = df[split:,1:]
test_y = df[split:,0]

print split
print train_x.shape,train_y.shape
print test_x.shape,test_y.shape

33600
(33600, 784) (33600,)
(8400, 784) (8400,)


In [7]:
train_y = np_utils.to_categorical(train_y)
test_y = np_utils.to_categorical(test_y)
print train_y.shape,test_y.shape
print "-----"
train_x = train_x.reshape((-1,28,28,1))
test_x = test_x.reshape((-1,28,28,1))
print train_x.shape,test_x.shape

(33600, 10) (8400, 10)
-----
(33600, 28, 28, 1) (8400, 28, 28, 1)


In [10]:
model = Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(28,28,1)))
model.add(Activation('tanh'))
model.add(Convolution2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(16,(3,3)))
model.add(Activation('tanh'))
model.add(Flatten())
model.add(Dropout(0.20))
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 16)        4624      
_________________________________________________________________
activation_6 (Activation)    (None, 10, 10, 16)        0         
__________

In [11]:
model.fit(train_x,train_y,verbose=2,epochs=10,validation_data=(test_x,test_y))

Train on 33600 samples, validate on 8400 samples
Epoch 1/10
 - 87s - loss: 0.1908 - acc: 0.9405 - val_loss: 0.0657 - val_acc: 0.9775
Epoch 2/10
 - 81s - loss: 0.0772 - acc: 0.9760 - val_loss: 0.0528 - val_acc: 0.9824
Epoch 3/10
 - 83s - loss: 0.0591 - acc: 0.9818 - val_loss: 0.0729 - val_acc: 0.9808
Epoch 4/10
 - 85s - loss: 0.0501 - acc: 0.9847 - val_loss: 0.0552 - val_acc: 0.9848
Epoch 5/10
 - 84s - loss: 0.0474 - acc: 0.9855 - val_loss: 0.0666 - val_acc: 0.9821
Epoch 6/10
 - 82s - loss: 0.0407 - acc: 0.9874 - val_loss: 0.0588 - val_acc: 0.9843
Epoch 7/10
 - 83s - loss: 0.0365 - acc: 0.9885 - val_loss: 0.0667 - val_acc: 0.9833
Epoch 8/10
 - 80s - loss: 0.0345 - acc: 0.9896 - val_loss: 0.0757 - val_acc: 0.9823
Epoch 9/10
 - 82s - loss: 0.0338 - acc: 0.9895 - val_loss: 0.0539 - val_acc: 0.9869
Epoch 10/10
 - 80s - loss: 0.0293 - acc: 0.9908 - val_loss: 0.0722 - val_acc: 0.9854


In [13]:
print test_file.shape
new_test_file = test_file.reshape((-1,28,28,1))
print new_test_file.shape

(28000, 784)
(28000, 28, 28, 1)


In [14]:
predictions = model.predict(new_test_file,verbose=2)

In [16]:
print len(predictions)
print predictions[0]

28000
[  3.88814646e-16   1.30857524e-17   1.00000000e+00   2.80908673e-12
   3.03726394e-17   1.20465970e-19   5.38580431e-20   4.10582999e-13
   1.38271379e-13   6.89280176e-14]


In [18]:
max_indices = []
for ix in predictions:
    index, value = max(enumerate(ix), key=operator.itemgetter(1))
    max_indices.append(index)

In [21]:
length = len(max_indices)
print length
counter = range(1,length+1)
print counter[:10]

28000
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [22]:
Dataframe = {
    'ImageId':counter,
    'Label':max_indices
}

In [23]:
output = pd.DataFrame(data=Dataframe)

In [24]:
output.to_csv('final.csv',sep=',',index=False)

In [25]:
#gave 98.38 accuracy 